#  Multi-step workflow executed by an ai agent to solve complex problems made using IBM Watsonx, beeai and DuckDuckGo search
## Create a concise 1-page market report for {company_name} that summarizes the research, competitive analysis, and highlights the market opportunities.

#### BeeAI Workflows
The agent's behavior is defined through workflow steps and the transitions between them. You can think of a Workflow as a graph that outlines the agent's behavior.
#### Basics of Workflows

The main components of a BeeAI workflow are state, defined as a Pydantic model, and steps, which are Python functions.

- State: Think of state as structured memory that the workflow can read from and write to during execution. It holds the data that flows through the workflow.
- Steps: These are the functional components of the workflow, connecting together to perform the agent’s actions.
#### A Multi-Step Workflow with Tools

Now that you understand the basic components of a Workflow, let’s explore the power of BeeAI Workflows by building a simple web search agent.

This agent creates a search query based on an input question, runs the query to retrieve search results, and then generates an answer to the question based on the results.

Let’s begin by importing the necessary modules.

In [1]:
# all needed imports
from pydantic import Field
from pydantic import BaseModel, ValidationError
from beeai_framework.workflows import Workflow, WorkflowError
from beeai_framework.backend import ChatModel, ChatModelOutput, ChatModelStructureOutput, UserMessage
from beeai_framework.template import PromptTemplate, PromptTemplateInput
from beeai import Bee # Tool  # BeeHive,
from beeai_framework.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
import json
from typing import Any
from beeai_framework.agents.react import ReActAgent, ReActAgentRunOutput
from beeai_framework.backend import ChatModel
from beeai_framework.adapters.watsonx import WatsonxChatModel
from beeai_framework.emitter import Emitter, EmitterOptions, EventMeta
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.backend import ChatModel, ChatModelOutput, UserMessage
from beeai_framework.adapters.watsonx import WatsonxChatModel
from dotenv import load_dotenv
import os

Next, we can define our workflow State.

In this case, the question field is required when instantiating the State. The other fields, search_results and answer, are optional during construction (defaulting to None), but they will be populated by the workflow steps as the execution progresses.

In [2]:
# Workflow State
class SearchAgentState(BaseModel):
    question: str
    search_results: str | None = None
    answer: str | None = None

Next, we define the ChatModel instance that will handle interaction with our LLM. For this example, we'll use IBM Granite 3.1 8B via Ollama. This model will be used to process the search query and generate answers based on the retrieved results.

In [3]:
# Load environment variables from .env file
load_dotenv()
# Required
WATSONX_URL=os.getenv("WATSONX_URL")
WATSONX_API_URL = os.getenv("WATSONX_URL")
WATSONX_API_KEY=os.getenv("WATSONX_API_KEY")
WATSONX_APIKEY=os.getenv("WATSONX_API_KEY")
WX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_PROJECT_ID=os.getenv("PROJECT_ID")

# Create a ChatModel to interface with ibm/granite-3-8b-instruct from watsonx
model = ChatModel.from_name(
    "watsonx:ibm/granite-3-8b-instruct",
    options={
        "project_id": WATSONX_PROJECT_ID,
        "api_key": WATSONX_API_KEY,
        "api_base": WATSONX_API_URL,
    },
)


Since this is a web search agent, we need a way to run web searches. For that, we'll use the DuckDuckGo

In [4]:
# Web search tool
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
search_tool=[DuckDuckGoSearchTool()]

In this workflow, we make extensive use of PromptTemplates and structured outputs.

Here, we define the various templates, input schemas, and structured output schemas that are essential for implementing the agent. These templates will allow us to generate the search query and structure the results in a way that the agent can process effectively.

In [5]:
# PromptTemplate Input Schemas
class QuestionInput(BaseModel):
    question: str


class SearchRAGInput(BaseModel):
    question: str
    search_results: str


# Prompt Templates
search_query_template = PromptTemplate(
    PromptTemplateInput(
        schema=QuestionInput,
        template="""Convert the following question into a concise, effective web search query using keywords and operators for accuracy.
Question: {{question}}""",
    )
)

search_rag_template = PromptTemplate(
    PromptTemplateInput(
        schema=SearchRAGInput,
        template="""Search results:
{{search_results}}

Question: {{question}}
Provide a concise answer based on the search results provided. If the results are irrelevant or insufficient, say 'I don't know.' Avoid phrases such as 'According to the results...'.""",
    )
)


# Structured output Schemas
class WebSearchQuery(BaseModel):
    query: str = Field(description="The web search query.")

Now, we can define the first step of the workflow, named web_search.

In this step:

- The LLM is prompted to generate an effective search query using the search_query_template.
- The generated search query is then used to run a web search via the search tool (Duckduckgo).
- The search results are stored in the search_results field of the workflow state.
- Finally, the step returns generate_answer, passing control to the next step, named generate_answer.

In [6]:
async def web_search(state: SearchAgentState) -> str:
    print("Step: ", "web_search")
    # Generate a search query
    prompt = search_query_template.render(QuestionInput(question=state.question))
    response: ChatModelStructureOutput = await model.create_structure(
        schema=WebSearchQuery, messages=[UserMessage(prompt)]
    )

    # Run search and store results in state
    try:
        #state.search_results = str(search_tool.run(response.object["query"]))
        duckduckgo_tool = DuckDuckGoSearchRun()
        state.search_results = duckduckgo_tool.invoke(response.object["query"])
    except Exception:
        print("Search tool failed! Agent will answer from memory.")
        state.search_results = "No search results available."

    return "generate_answer"

The next step in the workflow is generate_answer.

This step:

- Takes the question and search_results from the workflow state.
- Uses the search_rag_template to generate an answer based on the provided data.
- The generated answer is stored in the workflow state.
- Finally, the workflow ends by returning Workflow.END, signaling the completion of the agent’s task.

In [7]:
async def generate_answer(state: SearchAgentState) -> str:
    print("Step: ", "generate_answer")
    # Generate answer based on question and search results from previous step.
    prompt = search_rag_template.render(
        SearchRAGInput(question=state.question, search_results=state.search_results or "No results available.")
    )
    output: ChatModelOutput = await model.create(messages=[UserMessage(prompt)])

    # Store answer in state
    state.answer = output.get_text_content()
    return Workflow.END

Finally, we define the overall workflow and add the steps we developed earlier. This combines everything into a cohesive agent that can perform web searches and generate answers.

## Specify the name of the company for which you want the market report

In [8]:
company = "HDFC Bank"

query = "Create a concise 1-page market report for company "+ company + \
"that summarizes the research, competitive analysis, and highlights the market opportunities."

try:
    # Define the structure of the workflow graph
    search_agent_workflow = Workflow(schema=SearchAgentState, name="WebSearchAgent")
    search_agent_workflow.add_step("web_search", web_search)
    search_agent_workflow.add_step("generate_answer", generate_answer)

    # Execute the workflow
    search_response = await search_agent_workflow.run(
        SearchAgentState(question=query)
    )

    print("*****")
    print("Question: ", search_response.state.question)
    print("Answer: ", search_response.state.answer)

except WorkflowError:
    traceback.print_exc()
except ValidationError:
    traceback.print_exc()

Step:  web_search
Step:  generate_answer
*****
Question:  Create a concise 1-page market report for company HDFC Bankthat summarizes the research, competitive analysis, and highlights the market opportunities.
Answer:  **Market Report: HDFC Bank**

**Executive Summary:**

HDFC Bank, a leading player in the Indian banking sector, reported a 2% miss in PAT, reaching Rs167bn (1.8% RoA) in the latest quarter. The decline was primarily due to slower credit growth and higher provisions. Despite this, deposit growth remained robust at 16% YoY/2.5% QoQ, bolstered by a QoQ increase of Rs0.6trn. Credit growth, however, dipped to a new low of 3% YoY, reflecting the bank's focus on Loan-to-Deposit Ratio (LDR) management.

**Competitive Analysis:**

HDFC Bank's competitive strategies and market positioning are noteworthy. The bank continually adapts to evolving customer demands and leverages technological capabilities, ensuring future growth and excellent service delivery. Brokerages like Jefferies